# 数据挖掘

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import seaborn as sns
from datetime import datetime

## 数据转换

- 数据挖掘（机器学习）前，数据需要先过滤、整理、转换。
- 在[数据探索](data-exploration.ipynb)阶段，发现[测试数据](test_data.xls)某些字段需要额外处理。

字段 | 分类 | 处理方式
---|---|---
ship_name | 杂讯 | 丢弃
ship_mobile | 类别特征 | 重新编码
order_id | 无关分析 | 丢弃
total_amount | 数值特征 | ~~剔除异常值(0)，转成对数~~ 稍后决定
final_amount | 重复信息 | 丢弃
pay_status | 类别特征 | 使用
ship_status | 类别特征 | 使用
is_delivery | 无识别度 | 丢弃
try_order | 无识别度 | 丢弃
erp_order_id | 数值特征 | {0, 1}
erp_status | 数值特征 | 重新编码
createtime | 数值特征 | 转换成日期
last_modified | 数值特征 | 转换成日期
payment | 类别特征 | 重新编码
shipping_id | 无识别度 | 丢弃
shipping | 类别特征 | 重新编码
member_id | 类别特征 | 重新编码
promotion_type | 类别特征 | 重新编码
group_id | 类别特征 | 重新编码
groupOn_id | 类别特征 | 重新编码
is_leader | 数值特征 | {0, 1}
is_prepare | 无识别度 | 丢弃
group_num | 数值特征 | {0, 3}
status | 类别特征 | 重新编码
confirm_delivery | 类别特征 | 重新编码
confirm | 无识别度 | 丢弃
ship_area | 类别特征 | 抽取区码
weight | 数值特征 | ~~剔除异常值(0)，转成对数~~ 稍后决定
tostr | 无关分析 | 丢弃
itemnum | 数值特征 | 使用
ip | 杂讯 | 丢弃
ship_addr | 重复信息 | 丢弃
ship_zip | 无关分析 | 丢弃
ship_tel | 重复信息 | 丢弃
ship_email | 无识别度 | 丢弃
ship_time | 类别特征 | 抽取时段，重新编码
cost_item | 数值特征 | ~~剔除异常值(0)，转成对数~~ 稍后决定
is_tax | 数值特征 | {False, True}
tax_type | 类别特征 | 重新编码
cost_tax | 无识别度 | 丢弃
is_protect | 无识别度 | 丢弃
cost_protect | 无识别度 | 丢弃
cost_payment | 无识别度 | 丢弃
currency | 无识别度 | 丢弃
cur_rate | 无识别度 | 丢弃
advance | 数值特征 | ~~剔除异常值(0)，转成对数~~ 稍后决定
score_u | 数值特征 | ~~剔除异常值(0)，转成对数~~ 稍后决定
score_g | 数值特征 | ~~剔除异常值(0)，转成对数~~ 稍后决定
discount | 数值特征 | 转成正数
pmt_goods | 数值特征 | 使用
pmt_order | 数值特征 | 使用
payed | 杂讯 | 丢弃
disabled | 无识别度 | 丢弃
displayonsite | 数值特征 | {False, True}
mark_type | 无识别度 | 丢弃
cost_freight | 数值特征 | 使用
extend | 无识别度 | 丢弃
order_refer | 无识别度 | 丢弃
addon | 杂讯 | 丢弃
source | 类别特征 | 重新编码
city_link | 数值特征 | {0, 1, 2}
verify_city_link | 数值特征 | {-1，0，1，2}
fuzzy | 数值特征 | {0,1}
longitude | 数值特征 | ~~剔除异常值~~ 稍后决定
latitude | 数值特征 | ~~剔除异常值~~ 稍后决定
scalping | 数值特征 | {False, True}
out_trade_no | 无关分析 | 丢弃
autoSendErp | 数值特征 | {False,True}
autoSendErpStatus | 类别特征 | {0, 1, 2}
string_tag_id | 杂讯 | 丢弃
lang | 类别特征 | 重新编码
delivery_note_price | 类别特征 | 重新编码

### 步骤一：过滤字段

In [3]:
# 选取需要的字段
cols = ['ship_mobile', 'total_amount', 'pay_status', 'ship_status', 'erp_order_id', 'erp_status',
        'createtime', 'last_modified', 'payment', 'shipping', 'member_id', 'promotion_type',
        'group_id', 'groupOn_id', 'is_leader', 'group_num', 'status', 'confirm_delivery', 'ship_area',
        'weight', 'itemnum', 'ship_time', 'cost_item', 'is_tax', 'tax_type', 'advance',
        'score_u', 'score_g', 'discount', 'pmt_goods', 'pmt_order', 'displayonsite', 'cost_freight',
        'source', 'city_link', 'verify_city_link', 'fuzzy', 'longitude', 'latitude', 'scalping',
        'autoSendErp', 'autoSendErpStatus', 'lang', 'delivery_note_price']

df = pd.read_excel('test_data.xls', usecols=cols)
df.shape

(5881, 44)

### 步骤二：转换格式

In [4]:
# 函数：重新编码，转换level
def levelize(values):
    items = values.unique()
    mapping = dict((item, idx) for idx, item in enumerate(items))
    return values.map(mapping)

# 直接编码
cols = ['ship_mobile', 'erp_status', 'payment', 'shipping', 'member_id', 'promotion_type', 'group_id',
        'groupOn_id', 'status', 'confirm_delivery', 'tax_type', 'source', 'lang', 'delivery_note_price']

for col in cols:
    df[col] = levelize(df[col])

In [5]:
# 抽取区码
df.ship_area = df.ship_area.map(lambda area: int(area.split(':')[-1]))

In [6]:
# 抽取时段，重新编码
def time2code(t):
    mapping = {
        '08:00-12:00': 0,
        '14:00-18:00': 1,
        '18:00-21:00': 2,
        '14:00-21:00': 3}
    return mapping.get(t, 4)

df.ship_time = df.ship_time.fillna('??,??').map(lambda x: time2code(x.split(',')[1]))

In [7]:
# 将时间戳转换成日期
df.createtime = df.createtime.map(datetime.fromtimestamp).map(lambda ts: ts.dayofyear)
df.last_modified = df.last_modified.map(datetime.fromtimestamp).map(lambda ts: ts.dayofyear)

In [8]:
# 折扣：转成正数
df.discount = df.discount.map(abs)

### 步骤三：去除异常值

根据[箱形图](http://baike.baidu.com/link?url=p5iilyxKfj00v8yBe7AkHusrO94ulEySuxDyqefi_6Qk34eSjJDe0MYvF2ZWFY1ke_nRy4aI8svvxKttWPgD1ti0vL7DW6nTGeFwlVlnwDKzFojUsp32aaQ6km01wIa9Rps-Va0ntNji-_kvHnNHzA_d3XTX-bpVspI3_jiFOM0CqZe0767L0SCo9qr9SyOR)定义，过滤极端异常值。

In [9]:
_='''# 定义异常阈值
outlier_thershold = 3

# 函数：选出正常值
def choose_normal(values):
    Q1 = values.quantile(.25)
    Q3 = values.quantile(.75)
    IQR = Q3 - Q1
    upper_fence = Q3 + outlier_thershold * IQR
    lower_fence = Q1 - outlier_thershold * IQR
    return ((values > lower_fence) & (values < upper_fence))

log_normals = choose_normal(df.longitude) # 经度正常值
lat_normals = choose_normal(df.latitude)  # 纬度正常值
normals = log_normals & lat_normals       # 经、纬度正常值

# 去除地理位置异常值 (含离群值、缺失值)
df = df[normals]'''

In [10]:
_='''cols = ['total_amount', 'weight', 'cost_item', 'advance', 'score_u', 'score_g']

# 剔除异常值（0）,转成对数
for col in cols:
    df = df[df[col] > 0]
    df[col] = df[col].map(log)'''